In [ ]:
!pip install biopython scikit-learn matplotlib seaborn --quiet

import pandas as pd
import numpy as np
from Bio import SeqIO
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from random import shuffle
def load_sequences(file_paths, window=150, stride=75):
    sequences, labels = [], []
    for label, path in file_paths.items():
        for record in SeqIO.parse(path, "fasta"):
            seq = str(record.seq).replace("*", "")
            for i in range(0, len(seq) - window + 1, stride):
                frag = seq[i:i+window]
                if len(frag) == window:
                    sequences.append((frag, label))
    shuffle(sequences)
    fragments, labels = zip(*sequences)
    return list(fragments), list(labels)
def get_kmers(seq, k=3):
    return [seq[i:i+k] for i in range(len(seq)-k+1)]
def build_kmer_features(sequences, k=3):
    all_kmers = set()
    kmer_counts = []
    for seq in sequences:
        kmers = get_kmers(seq, k)
        count = Counter(kmers)
        kmer_counts.append(count)
        all_kmers.update(count)
    all_kmers = sorted(list(all_kmers))
    df = pd.DataFrame(0, index=np.arange(len(sequences)), columns=all_kmers)
    for i, count in enumerate(kmer_counts):
        for kmer, freq in count.items():
            df.iat[i, df.columns.get_loc(kmer)] = freq
    return df
file_paths = {
    'Zika': '/content/drive/MyDrive/sagar/Zika_200.fasta',
    'Ebola': '/content/drive/MyDrive/sagar/Ebola_200.fasta',
    'SARS_CoV_2': '/content/drive/MyDrive/sagar/SARS_CoV_2_200.fasta',
    'Influenza_A': '/content/drive/MyDrive/sagar/Influenza_A_200.fasta',
    'Influenza_B': '/content/drive/MyDrive/sagar/Influenza_B_200.fasta',
    'Tuberculosis': '/content/drive/MyDrive/sagar/Tuberculosis_200.fasta',
    'Human': '/content/drive/MyDrive/sagar/Human_200.fasta'
}
data, labels = load_sequences(file_paths, window=150, stride=75)
print(f"Augmented sequences: {len(data)}")
X = build_kmer_features(data, k=3)
y, class_names = pd.factorize(pd.Series(labels))
selector = SelectKBest(mutual_info_classif, k=600)
X_selected = selector.fit_transform(X, y)
X_train, X_test, y_train, y_test = train_test_split(
    X_selected, y, test_size=0.2, stratify=y, random_state=42
)
mnb_model = MultinomialNB(alpha=0.3)
mnb_model.fit(X_train, y_train)
y_pred = mnb_model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"\n Final Tuned Multinomial NaiveBaiyes Accuracy: {acc * 100:.2f}%")
print("\n Classification Report:\n")
print(classification_report(y_test, y_pred, target_names=class_names))

Augmented sequences: 6664

 Final Tuned MultinomialNB Accuracy: 96.25%

 Classification Report:

              precision    recall  f1-score   support

  SARS_CoV_2       1.00      1.00      1.00       551
Tuberculosis       1.00      0.95      0.97       173
 Influenza_A       0.93      0.89      0.91       164
       Human       0.92      1.00      0.96       291
        Zika       0.99      0.91      0.95       105
 Influenza_B       0.78      0.74      0.76        47
       Ebola       1.00      1.00      1.00         2

    accuracy                           0.96      1333
   macro avg       0.94      0.93      0.94      1333
weighted avg       0.96      0.96      0.96      1333

